In [ ]:
from risk_lib.data import get_factor_data
factor_data = get_factor_data()

In [ ]:
from risk_lib.stats import get_zscore

factor_data['zscore'] = get_zscore(factor_data.ret, factor_data.vol)
factor_data['zscore']

In [ ]:
sample = factor_data['zscore'].sel(vol_type=21, factor_name='QQQ').to_pandas().dropna()
plot_qq(sample)